## 1. 라이브러리 및 데이터 불러오기

In [1]:
# 분석에 필요한 라이브러리를 불러옵니다.
import pandas as pd
base_path = "C:/Users/gmlkd/data/energy/"

In [2]:
train = pd.read_csv(base_path + "train.csv", encoding="cp949")
test = pd.read_csv(base_path + "test.csv", encoding="cp949")
submission = pd.read_csv(base_path + "sample_submission.csv", encoding="cp949")

In [3]:
#Q. 122400의 의미는?
# 85 * 60 * 24 = 85일, 24시간 ,60개 건물
train

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0
3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0
4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
122395,60,2020-08-24 19,4114.368,27.8,2.3,68.0,0.0,0.7,1.0,1.0
122396,60,2020-08-24 20,3975.696,27.3,1.2,71.0,0.0,0.0,1.0,1.0
122397,60,2020-08-24 21,3572.208,27.3,1.8,71.0,0.0,0.0,1.0,1.0
122398,60,2020-08-24 22,3299.184,27.1,1.8,74.0,0.0,0.0,1.0,1.0


### column 별 설명
- num : 건물 번호 (1~60)
- date_time : 시간 단위
- 전력 사용량(kWh) : 시간당 전력사용량 (target) -> Regression
- 기온, 풍속, 습도, 강수량, 일조 : 기상 정보 (시간당)
- 비전기냉방설비운영, 태양광보유 : categorical feature (0/1) (건물 정보)

In [4]:
# column 이름 변경
train.columns = ["num", "date_time", "target", "temperature",
                "windspeed", "humidity", "precipitation",
                "insolation", "nelec_cool_flag", "solar_flag"]
test.columns = ["num", "date_time", "temperature",
                "windspeed", "humidity", "precipitation",
                "insolation", "nelec_cool_flag", "solar_flag"]

In [5]:
# Q. 10080은?
test 

,num,date_time,temperature,windspeed,humidity,precipitation,insolation,nelec_cool_flag,solar_flag
0,1,2020-08-25 00,27.8,1.5,74.0,0.0,0.0,NaN,NaN
1,1,2020-08-25 01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2020-08-25 02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2020-08-25 03,27.3,1.1,78.0,NaN,0.0,NaN,NaN
4,1,2020-08-25 04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
10075,60,2020-08-31 19,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10076,60,2020-08-31 20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10077,60,2020-08-31 21,27.9,4.1,68.0,NaN,0.0,1.0,1.0
10078,60,2020-08-31 22,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10080 entries, 0 to 10079
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   num              10080 non-null  int64  
 1   date_time        10080 non-null  object 
 2   temperature      3360 non-null   float64
 3   windspeed        3360 non-null   float64
 4   humidity         3360 non-null   float64
 5   precipitation    1680 non-null   float64
 6   insolation       3360 non-null   float64
 7   nelec_cool_flag  2296 non-null   float64
 8   solar_flag       1624 non-null   float64
dtypes: float64(7), int64(1), object(1)
memory usage: 708.9+ KB


### column별 설명 (test data)
- 기온, 풍속, 습도, 일조(hr, 3시간) : 3시간 당 측정한 값.
- 강수량(mm, 6시간) : 6시간 당 측정한 값.
- 비전기냉방설비운영, 태양광보유 : 건물 정보

## 2. 데이터 전처리

데이터 전처리할 때, 무조건 해야하는 것. **(No optional)**
- 결측치 처리 : 결측치를 채우거나, 지우거나해서 없애야 함.
- Categorical feature 처리 : object형 column은 무조건 숫자로 변환.

### 2-1. 결측치 처리


#### 건물 정보

In [7]:
#건물별로 '비전기냉방설비운영'과 '태양광보유'를 판단해 test set의 결측치를 보간해줍니다
train[["num", "nelec_cool_flag", "solar_flag"]].drop_duplicates()  # 중복 제거--> 건물별로 정보가 같다

,num,nelec_cool_flag,solar_flag
0,1,0.0,0.0
2040,2,1.0,0.0
4080,3,1.0,1.0
6120,4,1.0,1.0
8160,5,1.0,0.0
10200,6,0.0,0.0
12240,7,1.0,0.0
14280,8,1.0,1.0
16320,9,0.0,1.0
18360,10,1.0,0.0


### 좀 더 스마트한 방법으로 채워보기! 

In [8]:
building_info = train[["num", "nelec_cool_flag", "solar_flag"]].drop_duplicates()
building_info

,num,nelec_cool_flag,solar_flag
0,1,0.0,0.0
2040,2,1.0,0.0
4080,3,1.0,1.0
6120,4,1.0,1.0
8160,5,1.0,0.0
10200,6,0.0,0.0
12240,7,1.0,0.0
14280,8,1.0,1.0
16320,9,0.0,1.0
18360,10,1.0,0.0


In [9]:
# JOIN?
teste = test.drop(columns=['nelec_cool_flag', 'solar_flag'])
test = pd.merge(test, building_info, on=['num'])
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10080 entries, 0 to 10079
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   num                10080 non-null  int64  
 1   date_time          10080 non-null  object 
 2   temperature        3360 non-null   float64
 3   windspeed          3360 non-null   float64
 4   humidity           3360 non-null   float64
 5   precipitation      1680 non-null   float64
 6   insolation         3360 non-null   float64
 7   nelec_cool_flag_x  2296 non-null   float64
 8   solar_flag_x       1624 non-null   float64
 9   nelec_cool_flag_y  10080 non-null  float64
 10  solar_flag_y       10080 non-null  float64
dtypes: float64(9), int64(1), object(1)
memory usage: 945.0+ KB


#### 일조량 (insolation)

In [ ]:
# 가장 최근에 기록된 정보로 채워줍니다.


In [10]:
test.insolation.value_counts()

0.0    1872
0.1     120
3.0     102
0.6      83
0.2      74
0.9      71
0.5      63
1.0      62
1.2      60
0.3      57
1.6      55
2.7      54
1.8      53
0.4      49
1.1      49
2.8      46
2.4      45
1.4      42
1.3      40
2.2      40
0.7      39
0.8      37
2.0      36
2.9      35
2.1      29
1.5      29
2.6      26
1.9      26
1.7      25
2.3      24
2.5      17
Name: insolation, dtype: int64

In [ ]:
dummy = train.copy() # 실험용 train data를 하나 복사합니다.
# Q. dummy를 test data처럼 빈 칸을 뚫는다.
## (풍속, 기온, 일조량, 습도는 3칸마다 데이터가 있고, 강수량은 6칸마다 데이터가 있다.)
### make_train_nan 함수를 이용하여 dummy dataframe을 스펀지로 만든다.
# Hint : np.nan 사용하기

def make_train_nan(data, column, n):
    # data: 스펀지를 만들 원본 dataframe
    # columns: 스펀지를 만들 대상 columns
    # n: 몇번째마다 값을 둘 것인가 (3 or 6)
    for i in range(len(data)):
        
    
    
make_train_nan(dummy, [], 3)
make_train_nan(dummy, [], 6)

In [ ]:
def compare_interpolation_methods(data, column, methods, metric):
    # e.g. column == "windspeed"

    
    return error_dict

In [ ]:
from sklearn.metrics import mean_squared_error

# 1. interpolation 기법별로 에러(틀린 정도)를 계산합니다.
all_error_dict = {}

# 2. 계산 결과를 그래프로 그립니다.

In [ ]:
fig, axes = plt.subplots(1, 4, figsize = (18, 5))
for i in range(len(all_error_df.columns)):
    sns.lineplot(ax=axes[i], data=all_error_df.iloc[:, i].transpose(), sort=False)

In [ ]:
# 테스트 데이터에 결측치를 채워봅니다!
test.temperature = 
# 마지막 NaN을 채우기 위해서 linear interpolation을 적용해줍니다.
test.temperature = 

test.windspeed = 

test.humidity = 
# 마지막 NaN을 채우기 위해서 linear interpolation을 적용해줍니다.
test.humidity = 

test.precipitation = 

In [ ]:
#요일 변수를 추가해봅니다.
def weekday(x):


+ test 결측치를 채워봅니다.
+ test 데이터의 변수는 예보 데이터이며, 예보 데이터는 train 데이터의 기간에 생성된 것이기에 활용 가능합니다.

1) 평균으로 채우기


2) 최빈값으로 채우기


3) 근처(앞, 뒤)에 있는 값으로 채우기


4) interpolation

### 2-2. 상관관계 분석

In [ ]:
# 피처간 비슷한 변수들이 있는지 확인하기 위해 상관관계 분석을 수행합니다.


In [ ]:
# heatmap


In [ ]:
# 피처간 비슷한 변수들이 있는지 확인하기 위해 상관관계 분석을 수행합니다.
corr = []
for num in range(1,61):
    df = train[train.num==num]
    num_corr = df.corr()['target']
    num_corr = num_corr.drop(['num','nelec_cool_flag','solar_flag','target'])
    corr.append(num_corr)
corr_df = pd.concat(corr, axis=1).T
corr_df.index = list(range(1,61))

# 시각화
f, ax = plt.subplots(figsize=(20,8))
plt.title("Correlation between features and target", fontsize=15)
sns.heatmap(corr_df.T, cmap=sns.diverging_palette(240,10,as_cmap=True), ax=ax)
plt.xlabel('Building(num)')
plt.show()

### 2-3. 이상치 검출

1) IQR(Inter-Quantile Range)


2) Outlier Detection method (e.g. Isolation Forest)

### IQR 

In [ ]:
# 데이터 중에 전체 데이터의 패턴을 벗어나는 데이터가 있는지 확인합니다.
def get_outlier(data, column, weight=1.5):
    

    
    
    print("IQR은 %.2f이다." % IQR)
    print(f"lower_bound는 {lower_bound:.2f}이다.") # f-string
    print("upper_bound는 {:.2f}이다.".format(upper_bound))
    

    return outlier_idx

In [ ]:
selected_col = input("Which column ? ")
outlier_idx = get_outlier(train, selected_col)
train.loc[outlier_idx, :]

In [ ]:
# outlier detection
# anomaly detection

train.drop(outlier_idx)

### Isolation Forest (ML Model)

In [ ]:
from sklearn.ensemble import IsolationForest

X = train.drop(columns=["num", "date_time", "target"])

model = 

In [ ]:
# IsolationForest로 판단한 outlier들을 제거합니다


## 3. 예측 모델 구현

In [ ]:
# 전력사용량 예측을 위해 회귀 모델을 불러옵니다


In [ ]:
# 학습에 필요한 데이터셋을 만듭니다


In [ ]:
# 학습 및 모델 검증을 위해 KFold Cross Validation 기법을 사용합니다.


In [ ]:
# training

In [ ]:
# submission 파일 생성